In [ ]:
#### Packages ####
from pathlib import Path
from argparse import Namespace
from collections import Counter
import json
import os
import string
import time
import math
import numpy as np
import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd #just to open our file
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10, 10]
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#### Setting Direc
import os
os.chdir("/content/drive/My Drive/LMU-Statistik/SS22/WWC Hackathon for Social Good/wwcode-hackathon-2022/data")

In [75]:
# assign directory
directory = "climate-new-db-dataset/articles/"
# empty list for dataframes
dfs = []
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        #print(filename.split(".")[0])
        this_df = pd.read_json(f, lines=True)
        this_df["source"] = filename.split(".")[0]
        dfs.append(this_df)

# Combine all dfs
df = pd.concat(dfs)

In [78]:
# Overview of combined data
print(df.shape)
print(df.columns)
print(df.info())

(33362, 9)
Index(['id', 'body', 'headline', 'article_name', 'article_url',
       'date_published', 'article_length', 'date_uploaded', 'source'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33362 entries, 0 to 3618
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              0 non-null      float64
 1   body            33362 non-null  object 
 2   headline        33059 non-null  object 
 3   article_name    27674 non-null  object 
 4   article_url     33362 non-null  object 
 5   date_published  33072 non-null  object 
 6   article_length  33362 non-null  int64  
 7   date_uploaded   33362 non-null  object 
 8   source          33362 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 2.5+ MB
None


In [ ]:
# Export dataframe to csv
df.to_csv("climate-news-db-all.csv", encoding="utf-8")

## Explore Dataset

In [ ]:
# 